<a href="https://colab.research.google.com/github/satyamjh/DF/blob/master/News_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://medium.com/@robert.salgado/multiclass-text-classification-from-start-to-finish-f616a8642538
#https://github.com/robsalgado/personal_data_science_projects/blob/master/mulitclass_text_class/Multiclass%20Text%20Classification%20Notebook.ipynb
import pandas as pd
import numpy as np
from google.cloud import storage
import json
import tensorflow as tf
import pickle
print('TF Version:', tf.__version__)

TF Version: 2.3.0


In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = 'gcpt-294006'
!gcloud config set project {project_id}

KeyboardInterrupt: ignored

In [ ]:
#Load Json File Data
OBUCKET = (storage.Client(project='gcpt-294006')).get_bucket('gsfile')
blob = OBUCKET.get_blob('ML/df/news_classif.json')
jdata = json.loads(blob.download_as_string())
#print(jdata[1])

#Prepare Panda DataFrame
content, label = [],[]
for each in jdata:
    content.append(each['content'])
    label.append(each['annotation']['label'][0])
    
df = pd.DataFrame([content, label]).T
df.columns= ['content', 'label']
df.head()

,content,label
0,Unions representing workers at Turner Newall...,Business
1,"SPACE.com - TORONTO, Canada -- A second\team o...",SciTech
2,AP - A company founded by a chemistry research...,SciTech
3,AP - It's barely dawn when Mike Fitzpatrick st...,SciTech
4,AP - Southern California's smog-fighting agenc...,SciTech


In [ ]:
#Apply the function to preprocess the text. Tokenize, lower, expand contactions, lemmatize, remove punctuation, numbers and stop words
df['clean_text'] = df['content'].apply(process_text)
df.head()

,content,label,clean_text
0,Unions representing workers at Turner Newall...,Business,"[union, represent, worker, turner, newall, dis..."
1,"SPACE.com - TORONTO, Canada -- A second\team o...",SciTech,"[space.com, toronto, canada, second, team, roc..."
2,AP - A company founded by a chemistry research...,SciTech,"[company, chemistry, researcher, university, l..."
3,AP - It's barely dawn when Mike Fitzpatrick st...,SciTech,"[it is, barely, dawn, mike, fitzpatrick, start..."
4,AP - Southern California's smog-fighting agenc...,SciTech,"[southern, california's, smog-fighting, agency..."


In [ ]:
#Avg word count by category
df['word_count'] = df['content'].apply(word_count)
avg_wc = df.groupby('label').mean().reset_index()
avg_wc[['label','word_count']]

,label,word_count
0,Business,31.363684
1,SciTech,30.770000
2,Sports,31.348947
3,World,31.735789


In [ ]:
# LTSM Modelling
from keras.preprocessing.text import Tokenizer

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in Description(Length of Field).
MAX_SEQUENCE_LENGTH = 500
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['clean_text'].values)
word_index = tokenizer.word_index
print('\nFound %s unique tokens:' % len(word_index))
print('Unique Tokens:\n:', word_index )


Found 17342 unique tokens:
Unique Tokens:
: {'new': 1, 'reuters': 2, 'year': 3, 'company': 4, 'quot': 5, 'monday': 6, 'tuesday': 7, 'thursday': 8, 'wednesday': 9, 'win': 10, 'friday': 11, 'world': 12, 'state': 13, 'make': 14, 'week': 15, 'york': 16, 'report': 17, 'game': 18, 'million': 19, 'yesterday': 20, 'president': 21, 'plan': 22, 'day': 23, 'time': 24, 'price': 25, 'service': 26, 'corp': 27, 'sunday': 28, 'group': 29, 'government': 30, 'official': 31, 'announce': 32, 'oil': 33, 'second': 34, 'team': 35, 'people': 36, 'lead': 37, 'united': 38, 'month': 39, 'today': 40, 'end': 41, 'security': 42, 'percent': 43, 'season': 44, 'microsoft': 45, 'kill': 46, 'high': 47, 'software': 48, 'use': 49, 'market': 50, 'night': 51, 'saturday': 52, 'iraq': 53, 'minister': 54, 'share': 55, 'internet': 56, 'computer': 57, 'network': 58, 'help': 59, 'technology': 60, 'large': 61, 'run': 62, 'washington': 63, 'stock': 64, 'record': 65, 'big': 66, 'leader': 67, 'billion': 68, 'rise': 69, 'business': 7

In [ ]:
#import keras
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(df['clean_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (7600, 500)


In [ ]:
# Converting categorical labels to numbers.
Y = pd.get_dummies(df['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (7600, 4)


In [ ]:
# Train test split.
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6840, 500) (6840, 4)
(760, 500) (760, 4)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPool1D
from keras.optimizers import Adam
from tensorflow.keras.layers import SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

# Output Should be one of the 59 ResolverTeams
ResOut = df['label'].nunique()
model.add(Dense( ResOut , activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
print('\nShape of the Model:', model.output_shape )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 100)          5000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 500, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 5,080,804
Trainable params: 5,080,804
Non-trainable params: 0
_________________________________________________________________

Shape of the Model: (None, 4)


In [ ]:
epochs = 10
batch_size = 64

print(X_train.shape)
print(Y_train.shape)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

(6840, 500)
(6840, 4)
Epoch 1/10
97/97 [==============================] - 140s 1s/step - loss: 1.0455 - accuracy: 0.5712 - val_loss: 0.5816 - val_accuracy: 0.8216
Epoch 2/10
97/97 [==============================] - 139s 1s/step - loss: 0.3808 - accuracy: 0.8774 - val_loss: 0.4966 - val_accuracy: 0.8392
Epoch 3/10
97/97 [==============================] - 140s 1s/step - loss: 0.1798 - accuracy: 0.9443 - val_loss: 0.5112 - val_accuracy: 0.8421
Epoch 4/10
97/97 [==============================] - 139s 1s/step - loss: 0.0872 - accuracy: 0.9761 - val_loss: 0.6708 - val_accuracy: 0.8275
Epoch 5/10
97/97 [==============================] - 140s 1s/step - loss: 0.0529 - accuracy: 0.9859 - val_loss: 0.6129 - val_accuracy: 0.8246


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]*100))

24/24 [==============================] - 3s 121ms/step - loss: 0.5631 - accuracy: 0.8276
Test set
  Loss: 0.563
  Accuracy: 82.763


In [ ]:
model.fit(X, Y, epochs=1, batch_size=batch_size, validation_split=0.1, 
          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

107/107 [==============================] - 157s 1s/step - loss: 0.1380 - accuracy: 0.9604 - val_loss: 0.1456 - val_accuracy: 0.9618


In [ ]:
# Unique Categories
NCategory = sorted(df['label'].unique())

In [ ]:
# Prediction
NDescr = [ process_text("Amid mounting criticism, Kerala puts on hold controversial Police Act Amendment ordinance") ]
seq = tokenizer.texts_to_sequences(NDescr)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
predictotron = model.predict(padded)
print(NCategory[np.argmax(predictotron)])

World


In [ ]:
#Download Model
pickle.dump(model,open('model.pkl','wb'))
#Load Model Back
model=pickle.load(open('model.pkl','rb'))

NameError: ignored

In [ ]:
#https://medium.com/@ml_kid/how-to-save-our-model-to-google-drive-and-reuse-it-2c1028058cb2
from google.colab import drive
drive.mount('/content/drive')
#using model.save_weights() to save the weights of the model in HDF5 format
model.save("/content/drive/My Drive/ColabModel/TFModel/model.pkl")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import itertools, string, operator, re, unicodedata, nltk
from operator import itemgetter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from bs4 import BeautifulSoup
import numpy as np
from itertools import combinations
from gensim.models import Phrases
from collections import Counter
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

#Contraction map
c_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))
add_stop = ['', ' ', 'say', 's', 'u', 'ap', 'afp', '...', 'n', '\\']
stop_words = ENGLISH_STOP_WORDS.union(add_stop)
tokenizer_n = TweetTokenizer()
pattern = r"(?u)\b\w\w+\b" 
lemmatizer = WordNetLemmatizer()
punc = list(set(string.punctuation))

def casual_tokenizer(text): #Splits words on white spaces (leaves contractions intact) and splits out trailing punctuation
    tokens = tokenizer_n.tokenize(text)
    return tokens

#Function to replace the nltk pos tags with the corresponding wordnet pos tag to use the wordnet lemmatizer
def get_word_net_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemma_wordnet(tagged_text):
    final = []
    for word, tag in tagged_text:
        wordnet_tag = get_word_net_pos(tag)
        if wordnet_tag is None:
            final.append(lemmatizer.lemmatize(word))
        else:
            final.append(lemmatizer.lemmatize(word, pos=wordnet_tag))
    return final

def expandContractions(text, c_re=c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)

def remove_html(text):
    soup = BeautifulSoup(text, "html5lib")
    tags_del = soup.get_text()
    uni = unicodedata.normalize("NFKD", tags_del)
    bracket_del = re.sub(r'\[.*?\]', '  ', uni)
    apostrphe = re.sub('’', "'", bracket_del)
    string = apostrphe.replace('\r','  ')
    string = string.replace('\n','  ')
    extra_space = re.sub(' +',' ', string)
    return extra_space

def process_text(text):
    soup = BeautifulSoup(text, "lxml")
    tags_del = soup.get_text()
    no_html = re.sub('<[^>]*>', '', tags_del)
    tokenized = casual_tokenizer(no_html)
    lower = [item.lower() for item in tokenized]
    decontract = [expandContractions(item, c_re=c_re) for item in lower]
    tagged = nltk.pos_tag(decontract)
    lemma = lemma_wordnet(tagged)
    no_num = [re.sub('[0-9]+', '', each) for each in lemma]
    no_punc = [w for w in no_num if w not in punc]
    no_stop = [w for w in no_punc if w not in stop_words]
    return no_stop

def word_count(text):
    return len(str(text).split(' '))

def word_freq(clean_text_list, top_n):
    """
    Word Frequency
    """
    flat = [item for sublist in clean_text_list for item in sublist]
    with_counts = Counter(flat)
    top = with_counts.most_common(top_n)
    word = [each[0] for each in top]
    num = [each[1] for each in top]
    return pd.DataFrame([word, num]).T

def word_freq_bigrams(clean_text_list, top_n):
    """
    Word Frequency With Bigrams
    """
    bigram_model = Phrases(clean_text_list, min_count=2, threshold=1)
    w_bigrams = list(bigram_model[clean_text_list])
    flat_w_bigrams = [item for sublist in w_bigrams for item in sublist]
    with_counts = Counter(flat_w_bigrams)
    top = with_counts.most_common(top_n)
    word = [each[0] for each in top]
    num = [each[1] for each in top]
    return pd.DataFrame([word, num]).T

def bigram_freq(clean_text_list, top_n):
    bigram_model = Phrases(clean_text_list, min_count=2, threshold=1)
    w_bigrams = list(bigram_model[clean_text_list])
    flat_w_bigrams = [item for sublist in w_bigrams for item in sublist]
    bigrams = []
    for each in flat_w_bigrams:
        if '_' in each:
            bigrams.append(each)
    counts = Counter(bigrams)
    top = counts.most_common(top_n)
    word = [each[0] for each in top]
    num = [each[1] for each in top]
    return pd.DataFrame([word, num]).T

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
